In [1]:
## Bring in needed mods
import numpy as np, pandas as pd

## List to checksum
checkpath = '/home/croth/Self-Filamentation_B3502_progeny/202105_Samples_for_Illumina_ForCullen.xlsx'

## Load in sample / phenotype info
checksums = pd.read_excel(checkpath)

## Append a new id
checksums['NewID'] = [ '6975-S%s_S'%str(int(s[2:])) for s in checksums['Sample ID']]

## view head
checksums.head()

,Sample ID,Strain,Species,ID,Notes,Phenotyping Code for QTL,NewID
0,SS01,NIH12,Cryptococcus neoformans,SS-I532,WT,N,6975-S1_S
1,SS02,NIH433,Cryptococcus neoformans,SS-I533,WT,N,6975-S2_S
2,SS12,I641,Cryptococcus deneoformans,SS-I641,Remaining random spore progeny from B3502 crosses,N,6975-S12_S
3,SS13,I643,Cryptococcus deneoformans,SS-I643,Remaining random spore progeny from B3502 crosses,N,6975-S13_S
4,SS14,I645,Cryptococcus deneoformans,SS-I645,Remaining random spore progeny from B3502 crosses,N,6975-S14_S


In [2]:
## View tail
checksums.tail()

,Sample ID,Strain,Species,ID,Notes,Phenotyping Code for QTL,NewID
20,SS30,B_29,Cryptococcus deneoformans,NaN,Progeny from B3502 crosses (from the original ...,L,6975-S30_S
21,SS31,B_30,Cryptococcus deneoformans,NaN,Progeny from B3502 crosses (from the original ...,N,6975-S31_S
22,SS32,B_31,Cryptococcus deneoformans,NaN,Progeny from B3502 crosses (from the original ...,L,6975-S32_S
23,SS33,B_32,Cryptococcus deneoformans,NaN,Progeny from B3502 crosses (from the original ...,N (with rare spots of T),6975-S33_S
24,SS34,B_33,Cryptococcus deneoformans,NaN,Progeny from B3502 crosses (from the original ...,L,6975-S34_S


In [3]:
## Check work
errormess = "Samples and strains are not unique"
assert checksums[['Sample ID','Strain']].drop_duplicates().shape[0] == checksums[['Sample ID','Strain']].shape[0], errormess

In [4]:
## Set path to reference on big duck
refpath = '/analysis/CROTH/Self-Filamentation_B3502_progeny/REF/FungiDB-48_CneoformansJEC21_Genome.fasta'

## set data path
datapath = '/analysis/CROTH/SELFFILAM/'

## set fastq path
fastqpath = datapath + 'FASTQ/'

## set sam path 
sampath = datapath+'SAM/ADDITIONAL_PROGENY_B3502/'

## set bam path
#bampath = datapath+'BAM/'

## set bamaddrg path
addrgpath = '/home/croth/bin/./bamaddrg -b'

## set command
command = 'bwa mem -a -M %s %s %s | samtools view -F 4 -b | samtools sort -o %s'

In [5]:
## gather samples and strain
samples = checksums.NewID.tolist()
strains = checksums.Strain.tolist()

In [12]:
## construct bam commands and take them ordred by size
bwa = np.array([command%(refpath,
                         fastqpath+f+'*_R1_*.fastq.gz',
                         fastqpath+f+'*_R2*.fastq.gz',
                         sampath + f+'-sm.bam\n') + '\n%s %s -s %s -r %s > %s\n\nrm %s\n\n'%(
                         addrgpath,sampath+f+'-sm.bam',strains[i],'%s_%s'%(strains[i],f),
                        sampath+f+'-smr.bam',sampath+f+'-sm.bam'
                ) + 'samtools index %s-smr.bam\n\n\n\n'%(sampath+f)
                for i,f in enumerate(samples)])

In [20]:
samix = [b.split('\n')[-5] for b in bwa]
assert len(samix) == len(bwa)

In [24]:
print('\n'.join(samix))

samtools index /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S1_S-smr.bam
samtools index /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S2_S-smr.bam
samtools index /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S12_S-smr.bam
samtools index /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S13_S-smr.bam
samtools index /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S14_S-smr.bam
samtools index /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S15_S-smr.bam
samtools index /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S16_S-smr.bam
samtools index /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S17_S-smr.bam
samtools index /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S18_S-smr.bam
samtools index /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S19_S-smr.bam
samtools index /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S20_S-smr.bam
samtools index /analysi

In [13]:
## view first command
print(bwa[0])

bwa mem -a -M /analysis/CROTH/Self-Filamentation_B3502_progeny/REF/FungiDB-48_CneoformansJEC21_Genome.fasta /analysis/CROTH/SELFFILAM/FASTQ/6975-S1_S*_R1_*.fastq.gz /analysis/CROTH/SELFFILAM/FASTQ/6975-S1_S*_R2*.fastq.gz | samtools view -F 4 -b | samtools sort -o /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S1_S-sm.bam

/home/croth/bin/./bamaddrg -b /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S1_S-sm.bam -s NIH12 -r NIH12_6975-S1_S > /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S1_S-smr.bam

rm /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S1_S-sm.bam

samtools index /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S1_S-smr.bam






In [25]:
bwa

array(['bwa mem -a -M /analysis/CROTH/Self-Filamentation_B3502_progeny/REF/FungiDB-48_CneoformansJEC21_Genome.fasta /analysis/CROTH/SELFFILAM/FASTQ/6975-S1_S*_R1_*.fastq.gz /analysis/CROTH/SELFFILAM/FASTQ/6975-S1_S*_R2*.fastq.gz | samtools view -F 4 -b | samtools sort -o /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S1_S-sm.bam\n\n/home/croth/bin/./bamaddrg -b /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S1_S-sm.bam -s NIH12 -r NIH12_6975-S1_S > /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S1_S-smr.bam\n\nrm /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S1_S-sm.bam\n\nsamtools index /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S1_S-smr.bam\n\n\n\n',
       'bwa mem -a -M /analysis/CROTH/Self-Filamentation_B3502_progeny/REF/FungiDB-48_CneoformansJEC21_Genome.fasta /analysis/CROTH/SELFFILAM/FASTQ/6975-S2_S*_R1_*.fastq.gz /analysis/CROTH/SELFFILAM/FASTQ/6975-S2_S*_R2*.fastq.gz | samtools view -F 4 -b | samtools sort 

In [8]:
## View last of command
print(bwa[-1])

bwa mem -a -M /analysis/CROTH/Self-Filamentation_B3502_progeny/REF/FungiDB-48_CneoformansJEC21_Genome.fasta /analysis/CROTH/SELFFILAM/FASTQ/6975-S34_S*_R1_*.fastq.gz /analysis/CROTH/SELFFILAM/FASTQ/6975-S34_S*_R2*.fastq.gz | samtools view -F 4 -b | samtools sort -o /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S34_S-sm.bam

/home/croth/bin/./bamaddrg -b /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S34_S-sm.bam -s B_33 -r B_33_6975-S34_S > /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S34_S-smr.bam

rm /analysis/CROTH/SELFFILAM/SAM/ADDITIONAL_PROGENY_B3502/6975-S34_S-sm.bam

samtools index /analysis/CROTH/SELFFILAM/6975-S34_S-smr.bam






In [9]:
## Split progeny samples across 4 files
nfiles = 6
filenames = './run_ap_%s_bwa.sh'

c = 0 
for i in range(nfiles):
    filename = filenames%i
    
    thesefiles = bwa[i::nfiles]
    print(len(thesefiles))
    
    c = c + len(thesefiles)
    
    open(filename,'w').writelines(thesefiles)

## check work
assert len(bwa) == c

5
4
4
4
4
4


### NOTES
You will need to take the .sh files and run chmod +x *.sh to make them executables